In [1]:
import pecos as pc
import numpy as np
import matplotlib.pyplot as plt

In [40]:
# The stabilizer simulator
state = pc.simulators.SparseSim(3) # 3 qbs initialized to 000
state.print_stabs(); # stabilizer generators
print()
state.run_gate('CNOT', {(0,1)})
state.print_stabs();
print()
state.run_gate('X', {0})
state.print_stabs(print_destabs = True)

  ZII
  IZI
  IIZ

  ZII
  ZZI
  IIZ

 -ZII
 -ZZI
  IIZ
-------------------------------
  XXI
  IXI
  IIX


([' -ZII', ' -ZZI', '  IIZ'], ['  XXI', '  IXI', '  IIX'])

In [28]:
n_qbs = 10
plaquettes = [(3,4,5,6),(0,2,4,6),(1,2,5,6)] # correspond to stab set.
msmt_qbs = [7,8,9]
COR_TABLE = { 7: 3, 8: 0, 9: 1, 7+8: 4, 8+9: 2, 7+9: 5, 7+8+9: 6 }

# Generate circuits
def plaquette_qc(basis):
    qc = pc.circuits.QuantumCircuit()
    # qc.append('init |+>', {0,1,2,3,4,5,6})
    qc.append('init %s' % ('|0>' if basis == 'Z' else '|+>'), {7,8,9})
    for m_qb, plaquette in zip(msmt_qbs, plaquettes):
        for d_qb in plaquette:
            if basis == 'Z': qc.append('CNOT', {(d_qb, m_qb)})   # detect and correct X errors
            elif basis == 'X': qc.append('CNOT', {(m_qb, d_qb)}) # detect and correct Z errors
    qc.append('measure %s' % basis, {7,8,9}) # measure msmt qbs
    return qc

steane_x = plaquette_qc(basis='X')
steane_z = plaquette_qc(basis='Z')
circ_runner = pc.circuit_runners.Standard(seed=np.random.randint(1e9))
# depolar = pc.error_gens.DepolarGen(model_level='circuit')

state = pc.simulators.SparseSim(n_qbs)
# introduce some errors
state.run_gate('X', {5})

# Measure ancillas
x_msmt, _ = circ_runner.run(state, steane_x)
z_msmt, _ = circ_runner.run(state, steane_z)  
    
# Calculate syndromes, apply correction (if any)
x_syn = sum(list(*x_msmt.values()))
z_syn = sum(list(*z_msmt.values()))
if x_syn: state.run_gate('Z', {COR_TABLE[x_syn]})
if z_syn: state.run_gate('X', {COR_TABLE[z_syn]})

state.print_stabs();

  ZIZIZIZIZI
 -IZZIIZZIIZ
  ZZZIIIIIII
 -IIIZZZZZII
  ZIIZZIIIII
  IZIZIZIIII
  ZZIZIIZIII
  IIIXXXXIII
  XIXIXIXIII
  IXXIIXXIII
